In [ ]:
from collections import namedtuple, defaultdict, Counter
from glob import glob
from itertools import groupby
from operator import itemgetter
from os import remove
from pathlib import Path
import pickle

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel as LDA
from gensim.models import AuthorTopicModel as ATM

from matplotlib import pyplot as plt

import numpy as np

from pandas import DataFrame, read_csv, concat, Series, set_option

import pyLDAvis as ldavis
import pyLDAvis.gensim

import seaborn as sns

from sklearn.model_selection import train_test_split

import scipy
from scipy.spatial.distance import cdist
from scipy.sparse import coo_matrix as sparse_matrix

ldavis.enable_notebook()
%matplotlib notebook
set_option('display.max_colwidth', -1)
set_option('display.width', 100)
#%precision 4


Candidate = namedtuple('Candidate', ['iterations', 'num_topics'])



def get_i_t(filename):
    _, content, document_type = filename.split('-')
    i, t, _ = content.split('_')
    return int(i[1:]), int(t[1:]), document_type.split('.')[0]

get_texts = lambda df: df[target].str.split()
tobows = lambda df, d: concat([df['Anomaly_ID'], get_texts(df).apply(d.doc2bow)], axis=1)



isr2 = 2.0 ** -.5

def hellinger(x, y):
    return isr2 * np.sqrt(((np.sqrt(x) - np.sqrt(y)) ** 2).sum())



report_types = 'ISA', 'PFR', 'DPFR'

TEST_SIZE = 0.3
min_occurances = 2
target = 'GLOMUNSTEM'

%matplotlib notebook
%precision 4

BASEDIR = Path('../data')
OUT = Path('../output/atm/')



In [ ]:
with open(BASEDIR / 'processed_authors.csv') as fd:
    af = read_csv(fd)
af.shape
af.head()

In [ ]:
authors_by_type = {
    t: af[af.ReportType == t] 
    for t in af.ReportType.unique()
}

In [ ]:
documents_by_type = {
    t: read_csv(OUT / f'../norm_{t}.csv').dropna()
    for t in af.ReportType.unique()
}

In [ ]:
def get_train_test_and_vocab(documents, test_size, min_occurances=min_occurances):
    train, test = train_test_split(
        documents, test_size=test_size
    )
    train.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    vocab = Dictionary(
        train[target].str.split()
    )
    vocab.filter_extremes(no_below=min_occurances)

    return train, test, vocab


In [ ]:
def author_document_downselect(documents, authors, ANOMALY_LABEL='Anomaly_ID'):
    documents[ANOMALY_LABEL] = \
      'A' + documents[ANOMALY_LABEL].apply(str)

    idx = documents[ANOMALY_LABEL].isin(
        authors[ANOMALY_LABEL].unique()
    )

    documents = documents[idx]
    
    idx = authors[ANOMALY_LABEL].isin(
        documents[ANOMALY_LABEL].unique()
    )
    authors = authors[idx]
    return documents, authors


for document_type in documents_by_type:
    documents_by_type[document_type], authors_by_type[document_type] = \
        author_document_downselect(
          documents_by_type[document_type],
          authors_by_type[document_type]
    )


In [ ]:
model_by_type, train_documents_by_type, test_documents_by_type, dictionary_by_type = \
  dict(), dict(), dict(), dict()

train_author_table_by_type = dict()
test_author_table_by_type = dict()

def attribution_table(documents, relevent_authors):
    store = defaultdict(set)
    for idx, anomaly in documents.iterrows():
        authors_documents = relevent_authors[
            relevent_authors.Anomaly_ID == anomaly.Anomaly_ID
        ]

        for author in authors_documents.Users_ID:
            store[author].add(idx)

    return {k: list(v) for k, v in store.items()}


for document_type in documents_by_type:
    train_documents_by_type[document_type], \
    test_documents_by_type[document_type], \
    dictionary_by_type[document_type] = \
        get_train_test_and_vocab(documents_by_type[document_type], TEST_SIZE)

    train_author_table_by_type[document_type] = attribution_table(
        train_documents_by_type[document_type],
        authors_by_type[document_type]
    )

    test_author_table_by_type[document_type] = attribution_table(
        test_documents_by_type[document_type],
        authors_by_type[document_type]
    )

    print(f'{document_type: <4}')

In [ ]:
election_by_type = {
    'ISA' : Candidate(iterations=351, num_topics=40),
    'PFR' : Candidate(iterations=251, num_topics=25),
    'DPFR': Candidate(iterations=351, num_topics=25),
}



for document_type in election_by_type:
    c = election_by_type[document_type]

    corpus = tobows(
        train_documents_by_type[document_type],
        dictionary_by_type[document_type]
    )[target]

    atm = ATM(
        corpus=list(corpus),
        author2doc=train_author_table_by_type[document_type],
        num_topics=c.num_topics,
        iterations=c.iterations,
    )
    model_by_type[document_type] = atm


In [ ]:
def get_document_topics(model, doc_bow):

    gamma_chunk, sstats = model.inference(
        chunk=[doc_bow], author2doc=dict(), doc2author=dict(), 
        rhot=1.00,
        collect_sstats=True
    )

    return gamma_chunk


def get_model_author_topic_vectors(model):
    author_topic_vectors = np.zeros(
        (model.num_authors, model.num_topics)
    )

    for i, author in enumerate(model.id2author.values()):
        idx, scores = zip(*model.get_author_topics(author))
        author_topic_vectors[i, idx] = scores

    return author_topic_vectors


def get_sorted_authors(model, doc_bow, author_topic_vectors, metric=hellinger):
    doc_vector = get_document_topics(model, doc_bow)

    author_scores = np.argsort(
        cdist(doc_vector, author_topic_vectors, metric=metric)
    )

    contenders = [
        model.id2author[idx]
        for idx in author_scores[0]
    ]

    return contenders


In [ ]:
with open(BASEDIR / 'prs_signatures.csv') as fd:
    signatures = read_csv(fd)

signatures.head()
signatures.Users_ID = 'U' + signatures.Users_ID.apply(str)

In [ ]:
lookup = signatures.drop_duplicates(['Users_ID'])[['Users_ID', 'First_Name', 'Middle_Name', 'Last_Name']].set_index('Users_ID')

In [ ]:
document_type = 'ISA'

model = model_by_type[document_type]
author_topic_vectors = get_model_author_topic_vectors(model)
vocabulary = dictionary_by_type[document_type]

docs = [
    'gimbal drive motor friction',
    'rtg temperature charge capacity curiosity',
    'rtg temperature charge capacity voyager',
    'radiated image noise',
    'ir cooler temperature',
]

for d in docs:
    bow = vocabulary.doc2bow(d.split())

    candidates = get_sorted_authors(model, bow, author_topic_vectors)

    results = lookup.loc[Series(candidates)]
    display(d)
    display(results.head())


In [ ]:
with open(BASEDIR / 'prs.csv') as fd:
    prs = read_csv(fd, low_memory=False)[
        ['Anomaly_ID', 'Title', 'Description']
    ]
prs.Anomaly_ID = 'A' + prs.Anomaly_ID.apply(str)

In [ ]:
import re

In [ ]:

U = 'U76104'
P = authors_by_type[document_type][
    authors_by_type[document_type].Users_ID == U
]

prs[(prs.Anomaly_ID.isin(P.Anomaly_ID)) &
    #&((prs.Title.str.contains("(?i)"+re.sub(" ","|",d),na=False))|(prs.Description.str.contains("(?i)"+re.sub(" ","|",d),na=False)))
    True
   ][['Anomaly_ID', 'Title']].head()
